# General Imports

**Importing all libraries**

In [1]:
import numpy as np
import pandas as pd
import warnings

import plotly.graph_objects as go
from bayes_opt import BayesianOptimization
import statsmodels.api as sm

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.inspection import permutation_importance


Disable warnings

In [2]:
# Disable all warnings
warnings.filterwarnings("ignore")

# Enable warnings again
## warnings.filterwarnings("default")

Generate train and test data

In [3]:
# Read Data
df = pd.read_csv("data_finish_prep.csv")

# Select only the usefull part of the df
df.drop(columns=['verschil_Lengte',
               'verschil_Gewicht', 'verschil_TUG',
               'verschil_BMI', 'verschil_Conditie',
               'verschil_Lenigheid', 'verschil_Knijpkracht'], inplace=True)

# Drop NaN values
df.dropna(inplace=True)

# Define X and y
X = df.drop(columns=['verschil_6 MWT'])
y = df['verschil_6 MWT']

# Split data into train & test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Assuming you have X_train, y_train defined for regression
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return -scores.mean()

# MLR

**feature selection**

In [5]:
# Fit the Full Model
X =  sm.add_constant(X_train)  # Add constant term for the intercept
model = sm.OLS(y_train, X_train).fit()  # Fit the MLR model
best_model = model  # Initialize the best model
selected_features = X_train.columns.tolist()
best_features = X_train.columns.tolist()
best_features_rmse = np.sqrt(-cross_val_score(LinearRegression(), X_train, y_train, cv=10, scoring='neg_mean_squared_error').mean())

# Iteratively remove one feature at a time based on p-values
while True:
    # Compute p-values
    p_values = model.pvalues[1:]  # Exclude the constant term

    # Identify Insignificant Features
    insignificant_feature = p_values.idxmax()
    max_p_value = p_values.max()

    # Remove Insignificant Feature
    X = X.drop(insignificant_feature, axis=1)
    selected_features.remove(insignificant_feature)

    # When there a no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    model = sm.OLS(y_train, X).fit()
    
    # Evaluate Model Performance
    rmse = np.sqrt(-cross_val_score(LinearRegression(), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error').mean())

    # Check if the model performs better without the insignificant features
    if rmse <= best_features_rmse:
        best_features_rmse = rmse
        best_features = selected_features.copy()
        best_model = model

# Save results
best_model_mlr = LinearRegression().fit(X_train[best_features], y_train)
best_features_mlr = best_features

# Show results
print("Best features:", best_features)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features)))

Best features: ['GeboorteRelatief', 'SterfteRelatief', 'InBezitWoningcorporatie', 'PersonenautoSOverigeBrandstof', 'GeweldsEnSeksueleMisdrijven', 'AfstandTotVolkstuin', 'Mantelzorger']
Dropped features: ['TotaalDiefstalUitWoningSchuurED', 'AfstandTotSemiOpenbaarGroenTotaal', 'AfstandTotKinderdagverblijf', 'PersonenautoSPerHuishouden', 'AfstandTotPoppodium', 'UrenMantelzorgPerWeek', 'AfstandTotDagrecreatiefTerrein', 'AfstandTotOpenbaarGroenTotaal', 'AfstandTotSchool', 'AfstandTotZonnebank', 'AfstandTotBelangrijkOverstapstation', 'VernielingMisdrijfTegenOpenbareOrde', 'AfstandTotBuitenschoolseOpvang', 'ErvarenGezondheidGoedZeerGoed', 'BouwjaarVanaf2000', 'BeperkingInZien', 'ErnstigOvergewichtObesitas', 'AfstandTotOpenNatNatuurlijkTerrein', 'AfstandTotBrandweerkazerne', 'AfstandTotOpenNatTerreinTotaal', 'AfstandTotTreinstationsTotaal', 'kPersonenMetLaagsteInkomen', 'AfstandTotAttractie', 'VoldoetAanRichtlijnAlcoholgebruik', 'AfstandTotOvDagelLevensmiddelen', 'KLFinancieleDienstenOnroerend

*Er zijn geen hyperparameters om te optimaliseren.*

# Support Vector Machines

**Hyperparameter optimalisatie**

In [6]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    classifier = model(**params)
    
    # Calculating accuracy based on cross-validation
    scores = cross_val_score(classifier, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
    return scores.mean()

params_ranges = {
    'C': (0.1, 10),
    'kernel_int': (1, 4),
    'gamma': (0.001, 0.1)
}

kernel_mapping = {
    1: 'linear',
    2: 'rbf',
    3: 'poly',
    4: 'sigmoid'
}

# Example usage with SVM
model = SVR
model_bo = BayesianOptimization(f=lambda C, kernel_int, gamma:
                                    bo_params_generic(model, {
                                        'C': C,
                                        'kernel': kernel_mapping[int(kernel_int)],
                                        'gamma': gamma
                                    }, X_train, y_train),
                             pbounds=params_ranges)

results = model_bo.maximize(n_iter=40, init_points=10)
params = model_bo.max['params']

# Creating a model with the best hyperparameters
best_model_svm = model(
    C=params['C'],
    kernel=kernel_mapping[int(params['kernel_int'])],
    gamma=params['gamma']
)

# Fit the model
best_model_svm.fit(X_train, y_train)


|   iter    |  target   |     C     |   gamma   | kernel... |
-------------------------------------------------------------
| 1         | -4.578e+0 | 5.522     | 0.02113   | 2.579     |
| 2         | -4.575e+0 | 6.055     | 0.02225   | 2.558     |
| 3         | -4.521e+0 | 7.694     | 0.07436   | 3.869     |
| 4         | -4.553e+0 | 7.906     | 0.07924   | 2.346     |
| 5         | -4.525e+0 | 5.572     | 0.07804   | 3.487     |
| 6         | -4.53e+03 | 4.188     | 0.07188   | 3.914     |
| 7         | -4.558e+0 | 4.225     | 0.07391   | 2.514     |
| 8         | -4.555e+0 | 7.041     | 0.07779   | 2.709     |
| 9         | -4.571e+0 | 2.758     | 0.07276   | 2.395     |
| 10        | -4.551e+0 | 8.356     | 0.08802   | 2.586     |
| 11        | -4.521e+0 | 6.498     | 0.07917   | 3.987     |
| 12        | -4.575e+0 | 5.219     | 0.02876   | 3.994     |
| 13        | -4.508e+0 | 9.365     | 0.04472   | 1.898     |
| 14        | -4.506e+0 | 9.572     | 0.03906   | 1.708     |
| 15    

SVR(C=9.887090610768734, gamma=0.001, kernel='linear')

**Feature Selection**

In [7]:
# Get the accuracy score of the model with all features
best_accuracy = cross_val_score(best_model_svm, X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()
best_model = best_model_svm
selected_features = X_train.columns
best_features = selected_features

while len(selected_features) > 1:
    try:
        # Retrieve the coefficients from the SVM model
        coefficients = best_model_svm.coef_[0]

        # Sort features based on their coefficient magnitudes
        sorted_indices = np.argsort(np.abs(coefficients))
        selected_features = [X_train.columns[i] for i in sorted_indices[::-1]]

        # Remove the least important feature
        selected_features = selected_features[:-1]
        
        # Refit the linear SVM classifier
        best_model_svm_classification.fit(X_train[selected_features], y_train)
        
        # Evaluate Model Performance
        accuracy = cross_val_score(best_model_svm, X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

        # Check if the model performs better without the insignificant features
        if accuracy >= best_accuracy:
            best_accuracy = accuracy
            best_features = selected_features.copy()
            best_model = best_model_svm
    except:
        print("Scince the kernal is not linear, this feature selection is not possible") 
        break

# Save results
best_model_svm = best_model
best_features_svm = best_features

# Show results
print("Best features:", best_features_svm)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_svm)))

Scince the kernal is not linear, this feature selection is not possible
Best features: Index(['NederlandseAntillenEnAruba', 'GeboorteRelatief', 'SterfteRelatief',
       'GemiddeldeWoningwaarde', 'PercentageOnbewoond',
       'InBezitWoningcorporatie', 'InBezitOverigeVerhuurders',
       'EigendomOnbekend', 'BouwjaarVanaf2000', 'kPersonenMetLaagsteInkomen',
       'ALandbouwBosbouwEnVisserij', 'KLFinancieleDienstenOnroerendGoed',
       'PersonenautoSOverigeBrandstof', 'PersonenautoSPerHuishouden',
       'PersonenautoSNaarOppervlakte', 'AfstandTotHuisartsenpraktijk',
       'AfstandTotKinderdagverblijf', 'AfstandTotSchool', 'OppervlakteWater',
       'TotaalDiefstalUitWoningSchuurED',
       'VernielingMisdrijfTegenOpenbareOrde', 'GeweldsEnSeksueleMisdrijven',
       'AfstandTotHuisartsenpost', 'AfstandTotApotheek',
       'AfstandTotConsultatiebureau', 'AfstandTotOvDagelLevensmiddelen',
       'AfstandTotWarenhuis', 'AfstandTotCafeED', 'AfstandTotCafetariaED',
       'AfstandTotResta

# Random Forest

**Feature selection**

In [8]:
# Get the accuracy score of the model with all the features
rf = RandomForestRegressor(n_estimators=100, random_state=0).fit(X_train, y_train)
best_rf = rf
selected_features = X_train.columns.tolist()
best_features = selected_features
best_features_accuracy = cross_val_score(RandomForestRegressor(n_estimators=100, random_state=0), X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

while True:
    # Make DataFrame of feature impotances
    result = permutation_importance(
            rf, X_train[selected_features], y_train, n_repeats=10, random_state=42, n_jobs=-1
        )
    df_importances = pd.DataFrame({'feature': X_train[selected_features].columns, 'importance': result.importances_mean, 'std': result.importances_std})
    df_importances = df_importances.sort_values('importance')

    # Remove least important feature
    selected_features.remove(df_importances['feature'].iloc[0])

    # When there are no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    rf = RandomForestRegressor(n_estimators=100, random_state=0).fit(X_train[selected_features], y_train)

    # Evaluate Model Performance
    accuracy = cross_val_score(RandomForestRegressor(n_estimators=100, random_state=0), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

    # Check if the model performs better without the insignificant features
    if accuracy >= best_features_accuracy:
        best_features_accuracy = accuracy
        best_features = selected_features.copy()
        best_model = rf

# Save results
best_model_rf = best_model
best_features_rf = best_features

# Show results
print("Best features:", best_features_rf)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_rf)))

Best features: ['AfstandTotApotheek', 'AfstandTotBioscoop', 'AfstandTotAttractie']
Dropped features: ['TotaalDiefstalUitWoningSchuurED', 'AfstandTotSemiOpenbaarGroenTotaal', 'AfstandTotKinderdagverblijf', 'PersonenautoSPerHuishouden', 'AfstandTotPoppodium', 'UrenMantelzorgPerWeek', 'AfstandTotDagrecreatiefTerrein', 'AfstandTotOpenbaarGroenTotaal', 'AfstandTotSchool', 'AfstandTotZonnebank', 'AfstandTotBelangrijkOverstapstation', 'GeweldsEnSeksueleMisdrijven', 'VernielingMisdrijfTegenOpenbareOrde', 'AfstandTotBuitenschoolseOpvang', 'ErvarenGezondheidGoedZeerGoed', 'BouwjaarVanaf2000', 'BeperkingInZien', 'GeboorteRelatief', 'ErnstigOvergewichtObesitas', 'AfstandTotOpenNatNatuurlijkTerrein', 'AfstandTotBrandweerkazerne', 'AfstandTotOpenNatTerreinTotaal', 'AfstandTotTreinstationsTotaal', 'kPersonenMetLaagsteInkomen', 'VoldoetAanRichtlijnAlcoholgebruik', 'AfstandTotOvDagelLevensmiddelen', 'KLFinancieleDienstenOnroerendGoed', 'AfstandTotOpenDroogNatTerrein', 'AfstandTotVerblijfsrecreatiefTerr

**Hyperparameter optimalisatie**

In [9]:
def bo_params_rf(min_samples_split,max_depth,max_features,n_estimators):
    
    params = {
        'min_samples_split': round(min_samples_split),
        'max_depth': round(max_depth),
        'max_features': max_features,
        'n_estimators':round(n_estimators)
    }    
        
    clf = RandomForestRegressor(min_samples_split=int(params['min_samples_split']), 
                                 max_depth=int(params['max_depth']), 
                                 max_features=params['max_features'],
                                 n_estimators=int(params['n_estimators']), 
                                 bootstrap=False,
                                 n_jobs = -1)
    return cross_val_score(clf, X_train[best_features_rf], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

# Create Bayesian Optimization
rf_bo = BayesianOptimization(f=bo_params_rf, pbounds={
    'n_estimators': (100, 1000),
    'max_depth': (1, 20),
    'max_features': (0.1, 1),
    'min_samples_split': (2, 12)
})

# Bayesian optimization
results = rf_bo.maximize(n_iter=40, init_points=10)

# Getting best hyperparameters from bayesian optimization
params = rf_bo.max['params']
params['n_estimators']= round(params['n_estimators'])
params['min_samples_split']= round(params['min_samples_split'])
params['max_depth']= round(params['max_depth'])

# Making random forest model with the best hyperparameters
# criterion entropy is after the hyperparameter optimization since it's more accurate but increases training time
best_model_rf_classification = RandomForestRegressor(min_samples_split=params['min_samples_split'],
                                                max_depth=params['max_depth'],
                                                max_features=params['max_features'],
                                                n_estimators=params['n_estimators'],
                                                bootstrap=False)

best_model_rf_classification.fit(X_train[best_features_rf], y_train)

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | -4.481e+0 | 4.103     | 0.8689    | 9.584     | 904.9     |
| 2         | -4.481e+0 | 3.56      | 0.8826    | 9.711     | 831.6     |
| 3         | -4.485e+0 | 14.12     | 0.2246    | 4.379     | 544.1     |
| 4         | -4.475e+0 | 3.213     | 0.3754    | 8.735     | 140.7     |
| 5         | -4.484e+0 | 15.22     | 0.842     | 9.347     | 764.5     |
| 6         | -4.485e+0 | 13.04     | 0.6182    | 4.432     | 325.2     |
| 7         | -4.484e+0 | 18.05     | 0.9042    | 9.818     | 180.4     |
| 8         | -4.485e+0 | 17.01     | 0.3149    | 7.036     | 207.3     |
| 9         | -4.484e+0 | 16.41     | 0.9293    | 9.057     | 586.8     |
| 10        | -4.476e+0 | 2.585     | 0.385     | 11.6      | 446.5     |
| 11        | -4.476e+0 | 3.789     | 0.1702    | 9.295     | 141.9     |
| 12        | -4.518e+0 | 1.0       | 

RandomForestRegressor(bootstrap=False, max_depth=6,
                      max_features=0.23149460250575904, min_samples_split=11,
                      n_estimators=452)

# neural network

**Feature selection**

In [10]:
# Get the accuracy score of the model with all the features
mlp = MLPRegressor(random_state=0).fit(X_train, y_train)
best_model = mlp
selected_features = X_train.columns.tolist()
best_features = selected_features
best_features_accuracy = cross_val_score(MLPRegressor(random_state=0), X_train, y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

while True:
    # Retrieve the learned weights
    weights = mlp.coefs_[0]  # Weights connecting input features to the first hidden layer

    # Normalize the weights
    normalized_weights = np.abs(weights) / np.sum(np.abs(weights), axis=0)

    # Calculate feature importance
    feature_importance = np.mean(normalized_weights, axis=1)

    # Sort features based on their importance
    sorted_indices = np.argsort(feature_importance)
    selected_features = [selected_features[i] for i in sorted_indices[:-1]]

    # When there are no features left
    if len(selected_features) == 0:
        break
    
    # Refit the Model
    mlp = MLPRegressor(random_state=0).fit(X_train[selected_features], y_train)

    # Evaluate Model Performance
    accuracy = cross_val_score(MLPRegressor(random_state=0), X_train[selected_features], y_train, cv=10, scoring='neg_mean_squared_error', n_jobs=-1).mean()

    # Check if the model performs better without the insignificant features
    if accuracy >= best_features_accuracy:
        best_features_accuracy = accuracy
        best_features = selected_features.copy()
        best_model = mlp

# Save results
best_model_nn = best_model
best_features_nn = best_features

# Show results
print("Best features:", best_features_nn)
print("Dropped features:", list(set(X_train.columns.tolist()) - set(best_features_nn)))

Best features: ['NederlandseAntillenEnAruba', 'GeboorteRelatief', 'SterfteRelatief', 'GemiddeldeWoningwaarde', 'PercentageOnbewoond', 'InBezitWoningcorporatie', 'InBezitOverigeVerhuurders', 'EigendomOnbekend', 'BouwjaarVanaf2000', 'kPersonenMetLaagsteInkomen', 'ALandbouwBosbouwEnVisserij', 'KLFinancieleDienstenOnroerendGoed', 'PersonenautoSOverigeBrandstof', 'PersonenautoSPerHuishouden', 'PersonenautoSNaarOppervlakte', 'AfstandTotHuisartsenpraktijk', 'AfstandTotKinderdagverblijf', 'AfstandTotSchool', 'OppervlakteWater', 'TotaalDiefstalUitWoningSchuurED', 'VernielingMisdrijfTegenOpenbareOrde', 'GeweldsEnSeksueleMisdrijven', 'AfstandTotHuisartsenpost', 'AfstandTotApotheek', 'AfstandTotConsultatiebureau', 'AfstandTotOvDagelLevensmiddelen', 'AfstandTotWarenhuis', 'AfstandTotCafeED', 'AfstandTotCafetariaED', 'AfstandTotRestaurant', 'AfstandTotHotelED', 'AfstandTotBuitenschoolseOpvang', 'AfstandTotOpenbaarGroenTotaal', 'AfstandTotDagrecreatiefTerrein', 'AfstandTotBos', 'AfstandTotOpenNatTerr

**Hyperparameter optimalisatie**

In [11]:
def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Calculating rmse based on
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return scores.mean()

params_ranges = {
    'hidden_layer_sizes': (10, 100),
    'alpha': (0.0001, 0.1),
    'learning_rate_init': (0.001, 0.1),
    'max_iter': (100, 1000)
}

# Example usage with Neural Network
model = MLPRegressor
model_bo = BayesianOptimization(f=lambda hidden_layer_sizes, alpha, learning_rate_init, max_iter:
                                    bo_params_generic(model, {
                                        'hidden_layer_sizes': (int(round(hidden_layer_sizes)),),
                                        'alpha': alpha,
                                        'learning_rate_init': learning_rate_init,
                                        'max_iter': int(round(max_iter))
                                    }, X_train[best_features_nn], y_train),
                             pbounds=params_ranges)

results = model_bo.maximize(n_iter=40, init_points=10)
params = model_bo.max['params']

# Creating a model with the best hyperparameters
best_model_nn = model(
    hidden_layer_sizes=(int(round(params['hidden_layer_sizes'])),),
    alpha=params['alpha'],
    learning_rate_init=params['learning_rate_init'],
    max_iter=int(round(params['max_iter']))
)

# Fit the model
best_model_nn.fit(X_train[best_features_nn], y_train)

|   iter    |  target   |   alpha   | hidden... | learni... | max_iter  |
-------------------------------------------------------------------------
| 1         | -67.2     | 0.04307   | 66.21     | 0.007714  | 239.4     |
| 2         | -67.11    | 0.06746   | 13.27     | 0.04038   | 944.5     |
| 3         | -67.16    | 0.01971   | 94.58     | 0.05792   | 400.1     |
| 4         | -67.1     | 0.009786  | 95.8      | 0.05962   | 586.8     |
| 5         | -67.2     | 0.005891  | 18.67     | 0.08431   | 278.1     |
| 6         | -67.17    | 0.01697   | 70.04     | 0.02806   | 329.8     |
| 7         | -66.88    | 0.08656   | 10.71     | 0.05557   | 511.4     |
| 8         | -67.24    | 0.09568   | 64.14     | 0.005395  | 697.8     |
| 9         | -67.14    | 0.07081   | 84.7      | 0.06444   | 850.1     |
| 10        | -67.0     | 0.05967   | 46.61     | 0.01851   | 645.5     |
| 11        | -67.11    | 0.06614   | 10.86     | 0.08457   | 511.1     |
| 12        | -67.28    | 0.08851   | 

MLPRegressor(alpha=0.0865633552041844, hidden_layer_sizes=(11,),
             learning_rate_init=0.05557146485916377, max_iter=511)

# plotting the bar chart

**using train dataset**

In [12]:
regression_models = [
    {
        'name': 'Multiple Linear Regression',
        'model': best_model_mlr,
        'features': best_features_mlr
    },
    {
        'name': 'Random Forest',
        'model': best_model_rf,
        'features': best_features_rf
    },
    {
        'name': 'Neural Network',
        'model': best_model_nn,
        'features': best_features_nn
    },
    {
        'name': 'Support Vector Machine',
        'model': best_model_svm,
        'features': best_features_svm
    }
]

regression_stats_val = []

for model in regression_models:
    try:
        # Perform cross-validation and calculate predictions and metrics
        y_pred = cross_val_predict(model['model'], X_train[model['features']], y_train, cv=10)
        y_true = y_train
    except: # cv can't be higher then n_splits in the dt
        # Perform cross-validation and calculate predictions and metrics
        y_pred = cross_val_predict(model['model'], X_train[model['features']], y_train, cv=5)
        y_true = y_train

    # R2 Score
    r2 = r2_score(y_true, y_pred)

    # Adjusted R2 Score
    n = len(y_true)  # Number of samples
    p = X_test.shape[1]  # Number of predictors (features) in X
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    # Save stats
    regression_stats_val.append(
        {
            'model': model['name'],
            'mae': mean_absolute_error(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2,
            'adj_r2': adj_r2
        }
    )

# View stats
regression_stats_val

[{'model': 'Multiple Linear Regression',
  'mae': 51.80079528782675,
  'mse': 4574.852965355593,
  'rmse': 67.63765937224316,
  'r2': 0.002518915465752336,
  'adj_r2': -0.07567109116067239},
 {'model': 'Random Forest',
  'mae': 51.20904367269245,
  'mse': 4482.568726908269,
  'rmse': 66.95198822221988,
  'r2': 0.022640170279610627,
  'adj_r2': -0.05397258233005031},
 {'model': 'Neural Network',
  'mae': 51.38431198591982,
  'mse': 4523.164857569359,
  'rmse': 67.25447834582734,
  'r2': 0.013788766147855958,
  'adj_r2': -0.06351782553148344},
 {'model': 'Support Vector Machine',
  'mae': 51.1046341381457,
  'mse': 4504.0257791475715,
  'rmse': 67.11203900305497,
  'r2': 0.017961767738447376,
  'adj_r2': -0.05901771295394753}]

In [13]:
# Extracting model names and metric scores
model_names = [d['model'] for d in regression_stats_val]
metric_labels = list(regression_stats_val[0].keys())[1:]
metric_scores = {metric: [d[metric] for d in regression_stats_val] for metric in metric_labels}

# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison 6MWT',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()

**Evaluating on test data**

In [14]:
regression_models = [
    {
        'name': 'Multiple Linear Regression',
        'model': best_model_mlr,
        'features': best_features_mlr
    },
    {
        'name': 'Random Forest',
        'model': best_model_rf,
        'features': best_features_rf
    },
    {
        'name': 'Neural Network',
        'model': best_model_nn,
        'features': best_features_nn
    },
    {
        'name': 'Support Vector Machine',
        'model': best_model_svm,
        'features': best_features_svm
    }
]

regression_stats_test = []

for model in regression_models:
    # Make predictions on the test set
    y_pred = model['model'].predict(X_test[model['features']])

    # Defining Actual values
    y_true = y_test.copy()

    # R2 Score
    r2 = r2_score(y_true, y_pred)

    # Adjusted R2 Score
    n = len(y_true)  # Number of samples
    p = X_test.shape[1]  # Number of predictors (features) in X
    adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)

    # Save stats
    regression_stats_test.append(
        {
            'model': model['name'],
            'mae': mean_absolute_error(y_true, y_pred),
            'mse': mean_squared_error(y_true, y_pred),
            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),
            'r2': r2,
            'adj_r2': adj_r2
        }
    )

# View stats
regression_stats_test

[{'model': 'Multiple Linear Regression',
  'mae': 49.8918225194864,
  'mse': 4851.733018231057,
  'rmse': 69.65438262041418,
  'r2': 0.03263162368111683,
  'adj_r2': -0.3633671268587768},
 {'model': 'Random Forest',
  'mae': 50.78543381200227,
  'mse': 4848.774530620403,
  'rmse': 69.63314247267894,
  'r2': 0.03322150514110711,
  'adj_r2': -0.36253577345610055},
 {'model': 'Neural Network',
  'mae': 50.255410541596184,
  'mse': 4817.906865245726,
  'rmse': 69.41114366761094,
  'r2': 0.03937609015682242,
  'adj_r2': -0.35386176767371813},
 {'model': 'Support Vector Machine',
  'mae': 49.95468166287966,
  'mse': 4780.516023273689,
  'rmse': 69.14127582908554,
  'r2': 0.04683130625213716,
  'adj_r2': -0.34335470873236806}]

In [35]:
# Plotting the bar chart
fig = go.Figure()

for metric in metric_labels:
    fig.add_trace(go.Bar(
        x=model_names,
        y=metric_scores[metric],
        text=metric_scores[metric],  # Display the values on the bars
        textposition='auto',  # Position the text inside the bars
        name=metric
    ))

# Updating the layout
fig.update_layout(
    title='Evaluation Metrics Comparison 6MWT',
    xaxis_title='Models',
    yaxis_title='Scores',
    barmode='group'
)

# Display the plot
fig.show()


Naive bayes

In [16]:
# Defining Actual values
y_true = y_test.copy()

# Defining Predicted values
y_pred = np.full_like(y_test, y_train.mean())

# Mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

# Mean Squared Error
mse = mean_squared_error(y_true, y_pred)
print("Mean Squared Error:", mse)

# Root Mean Squared Error
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

# R2 Score
r2 = r2_score(y_true, y_pred)
print("R2 Score:", r2)

# Adjusted R2 Score
n = len(y_true)  # Number of samples
p = X_test.shape[1]  # Number of predictors (features) in X
adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)
print("Adjusted R2 Score:", adj_r2)

Mean Absolute Error: 50.91615513871266
Mean Squared Error: 5027.712127213051
Root Mean Squared Error: 70.90636168365326
R2 Score: -0.002456173665164929
Adjusted R2 Score: -0.4128183500193261


Save best model as Pickle

In [17]:
import pickle

# Save model
file_path = 'Saved Models/6 MWT.pkl'

with open(file_path, 'wb') as file:
    pickle.dump(best_model_mlr, file)

# Save features
with open('Saved Models/best_features_6 MWT.txt', 'w') as file:
    file.write(f'{best_features_mlr}')